## Multi-label Prediction
This notebooks does multi-label prediction on malaria dataset. It doesn't calculate bounding boxes (BB).

In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2

In [2]:
types = {"schizont": 2, "gametocyte": 3, "ring": 4, "trophozoite": 5, "red blood cell": 0, "leukocyte": 1}

typeArr = np.empty(shape=(6,),dtype="U32")

for k in types:
    typeArr[types[k],] = k
    
print(typeArr)

['red blood cell' 'leukocyte' 'schizont' 'gametocyte' 'ring' 'trophozoite']


In [3]:
trainingImages = tf.data.Dataset.list_files("malaria/training/*.png", shuffle=False)
testImages = tf.data.Dataset.list_files("malaria/test/*.png", shuffle=False)

print(len(trainingImages), len(testImages))

818 390


#### Data Ingestion Pipeline

In [4]:
BATCH_SIZE= 32
img_shape = (224,224,3) # height, width, channels
autoTune = tf.data.AUTOTUNE

In [5]:
def read_npy_file(item):
    data = np.load(item.numpy().decode('UTF-8'), allow_pickle=True, fix_imports=False)
    return data.astype("int32")

In [6]:
def decode_img(img: str, img_height: int, img_width: int):
    # convert the compressed string to a 3D uint8 tensor
    png = tf.io.decode_png(img, channels=3, dtype=tf.dtypes.uint8)
    
    # resize the image to the desired size
    return tf.image.resize(png, [img_height, img_width], method="bilinear", preserve_aspect_ratio=False, antialias=True)

In [7]:
def process_path(file_path):
    
    imgName = tf.strings.regex_replace(file_path, "training|test", "Y_nobb", replace_global=False)
    imgName = tf.strings.regex_replace(imgName, ".png", ".npy", replace_global=False)
    
    label = tf.py_function(read_npy_file, [imgName], [tf.float32])
    label = tf.reshape(label, [6,])
        
  # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img, img_shape[0], img_shape[1])
    
    return img, label

In [8]:
def configure_for_performance(ds, shuffle=True):
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=autoTune)
    return ds

In [9]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = trainingImages.map(process_path, num_parallel_calls=autoTune)
test_ds = testImages.map(process_path, num_parallel_calls=autoTune)

In [10]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  [1. 0. 0. 0. 1. 0.]


In [11]:
train_ds = configure_for_performance(train_ds)
test_ds = configure_for_performance(test_ds)

#### Create and load model

In [ ]:
num_gpus = len(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus)

if num_gpus>0:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.OneDeviceStrategy(device = "/device:CPU:0")

In [12]:
### Model parameters
EPOCHS = 50

num_classes = 6
lr = 1e-3

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_bin_acc', min_delta=1e-4, patience=10, verbose=0, restore_best_weights=True)
terminate = tf.keras.callbacks.TerminateOnNaN()

myCallbacks = [early_stop, terminate]

In [13]:
def createModel(lr, num_classes, img_shape):
    
    inputLayer = tf.keras.Input(shape=img_shape)
    
    res = tf.keras.applications.InceptionV3(include_top=False,weights="imagenet",
    input_shape=img_shape,
    pooling=None, classifier_activation=None)
    
    x = res(inputLayer)
    x = tf.keras.layers.GlobalMaxPooling2D(data_format="channels_last")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32, "relu", use_bias=True, kernel_initializer='glorot_normal')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(16, "relu", use_bias=True, kernel_initializer='glorot_normal')(x)
    x = tf.keras.layers.Dense(num_classes, "sigmoid", use_bias=True, kernel_initializer='glorot_normal')(x)

    model = tf.keras.Model(inputs=[inputLayer], outputs=[x])
    
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    met = tf.keras.metrics.BinaryAccuracy(name='bin_acc', dtype=None, threshold=0.5)
    model.compile(opt, loss="binary_crossentropy", metrics=[met])
    
    return model
    

In [14]:
with strategy.scope():
    model = createModel(lr, num_classes, img_shape)
    model.summary()
    model.fit(train_ds, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=test_ds, callbacks = myCallbacks)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                65568     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528   

In [16]:
with strategy.scope():
    model.fit(train_ds, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=test_ds, callbacks = myCallbacks)

Epoch 1/50
26/26 [==============================] - 33s 671ms/step - loss: 0.6805 - bin_acc: 0.7148 - val_loss: 101.3192 - val_bin_acc: 0.7299
Epoch 2/50
26/26 [==============================] - 22s 561ms/step - loss: 0.4342 - bin_acc: 0.8323 - val_loss: 4.1241 - val_bin_acc: 0.6534
Epoch 3/50
26/26 [==============================] - 24s 608ms/step - loss: 0.3819 - bin_acc: 0.8510 - val_loss: 3.1325 - val_bin_acc: 0.8299
Epoch 4/50
26/26 [==============================] - 24s 624ms/step - loss: 0.3616 - bin_acc: 0.8566 - val_loss: 0.8411 - val_bin_acc: 0.8308
Epoch 5/50
26/26 [==============================] - 22s 559ms/step - loss: 0.3356 - bin_acc: 0.8717 - val_loss: 4.3309 - val_bin_acc: 0.7333
Epoch 6/50
26/26 [==============================] - 22s 551ms/step - loss: 0.3225 - bin_acc: 0.8755 - val_loss: 0.4746 - val_bin_acc: 0.8415
Epoch 7/50
26/26 [==============================] - 22s 543ms/step - loss: 0.3250 - bin_acc: 0.8704 - val_loss: 1.2489 - val_bin_acc: 0.8380
Epoch 8/50


#### Evaluate

In [17]:
def prec_recall_fscore(tp,fp,fn,tn):
    
    if tp+fp == 0:
        prec = np.nan
    else:
        prec = np.round(tp/(tp+fp),3)
        
    if tp+fn == 0:
        recall = np.nan
    else:
        recall = np.round(tp/(tp+fn),3)
        
    if prec + recall == 0:
        fscore = np.nan
    else:
        fscore = np.round((2 * prec * recall)/(prec + recall),3)
    
    return prec, recall, fscore

In [18]:
def acc_prec_recall(res,gnd_truth, typeArr):
    
    """Outputs accuracy, precision, recall, fscore for each class in a multi-label problem
    
    Inputs: 
        res : 2-D boolean numpy array containing the model predictions
        gnd_truth: 2-D boolean numpy array containing ground truth
        typeArr: 1-D numpy array of strings in which the column of the class label corresponds to the column in the ground truth array
    
    Outputs:
        output: Python dictionary containing the accuracy, precision, recall, fscore for each class
        overall_acc: float value of the overall accuracy in the whole dataset
    """
        
    output = dict()
    overallOutput = dict()
        
    equal = res==gnd_truth
    
    # overall
    tp = np.sum((res==1) & (gnd_truth==1),axis=None)
    fp = np.sum((res==1) & (gnd_truth==0),axis=None)
    fn = np.sum((res==0) & (gnd_truth==1),axis=None)
    tn = np.sum((res==0) & (gnd_truth==0),axis=None)
    overallOutput["acc"] = np.round(np.sum(equal,axis=None)/np.multiply.reduce(equal.shape),3) # overall accuracy
    overallOutput["prec"], overallOutput["recall"], overallOutput["fscore"] = prec_recall_fscore(tp,fp,fn,tn)
    
    # per class
    for i in range(equal.shape[1]):
        output[typeArr[i]] = {"count": 0,"acc":0, "prec":0, "recall":0, "fscore" : 0}

        output[typeArr[i]]["count"] = np.sum(gnd_truth[:,i]==1, axis=None)

        tp = np.sum((res[:,i]==1) & (gnd_truth[:,i]==1),axis=None)
        fp = np.sum((res[:,i]==1) & (gnd_truth[:,i]==0),axis=None)
        fn = np.sum((res[:,i]==0) & (gnd_truth[:,i]==1),axis=None)
        tn = np.sum((res[:,i]==0) & (gnd_truth[:,i]==0),axis=None)

        output[typeArr[i]]["acc"] = np.round((tp+tn)/(tp+tn+fp+fn),3)
        output[typeArr[i]]["prec"], output[typeArr[i]]["recall"], output[typeArr[i]]["fscore"] = prec_recall_fscore(tp,fp,fn,tn)
        

    return output, overallOutput
    

In [19]:
testImages = tf.data.Dataset.list_files("malaria/test/*.png", shuffle=False)
test_ds = testImages.map(process_path, num_parallel_calls=autoTune)
test_ds = configure_for_performance(test_ds, shuffle=False)

pred = model.predict(test_ds, batch_size=BATCH_SIZE)
res = pred>=0.5

In [20]:
imgOrder = [f.numpy().decode('UTF-8') for f in testImages]

gnd_truth = np.zeros(shape=(len(imgOrder),num_classes),dtype="bool")

i=0
for file in imgOrder:
    gnd_truth[i,:] = np.load(file.replace("test","Y_nobb").replace(".png",".npy"),allow_pickle=True, fix_imports=False)
    i+=1

In [21]:
class_metrics, acc = acc_prec_recall(res,gnd_truth,typeArr)

for k in class_metrics:
    print(k, class_metrics[k])

red blood cell {'count': 390, 'acc': 1.0, 'prec': 1.0, 'recall': 1.0, 'fscore': 1.0}
leukocyte {'count': 33, 'acc': 0.897, 'prec': 0.37, 'recall': 0.303, 'fscore': 0.333}
schizont {'count': 49, 'acc': 0.879, 'prec': 0.562, 'recall': 0.184, 'fscore': 0.277}
gametocyte {'count': 45, 'acc': 0.885, 'prec': nan, 'recall': 0.0, 'fscore': nan}
ring {'count': 78, 'acc': 0.813, 'prec': 0.558, 'recall': 0.308, 'fscore': 0.397}
trophozoite {'count': 193, 'acc': 0.749, 'prec': 0.868, 'recall': 0.58, 'fscore': 0.695}


In [22]:
acc

{'acc': 0.871, 'prec': 0.901, 'recall': 0.692, 'fscore': 0.783}